In [1]:
!pip install albumentations==0.4.6

     |████████████████████████████████| 122kB 8.4MB/s 
     |████████████████████████████████| 952kB 42.4MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp37-none-any.whl size=65175 sha256=483564ad3e06f43b7ad0d949993d82dcf4440b7d4247afbfa44a8ec2416c29ca
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [2]:
import os

if not os.path.exists("/content/classification"):
  !unzip classification.zip

Archive:  classification.zip
   creating: classification/
   creating: classification/vechicles/
   creating: classification/vechicles/test/
   creating: classification/vechicles/test/bike/
  inflating: classification/vechicles/test/bike/0.jpg  
  inflating: classification/vechicles/test/bike/1.jpg  
  inflating: classification/vechicles/test/bike/2.jpg  
  inflating: classification/vechicles/test/bike/3.jpg  
  inflating: classification/vechicles/test/bike/4.jpg  
  inflating: classification/vechicles/test/bike/5.jpg  
  inflating: classification/vechicles/test/bike/6.jpg  
   creating: classification/vechicles/test/boat/
  inflating: classification/vechicles/test/boat/10.jpg  
  inflating: classification/vechicles/test/boat/11.jpg  
  inflating: classification/vechicles/test/boat/12.jpg  
  inflating: classification/vechicles/test/boat/13.jpg  
  inflating: classification/vechicles/test/boat/14.jpg  
  inflating: classification/vechicles/test/boat/15.jpg  
  inflating: classification

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms.functional as TF

import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader

import numpy as np

#augmentation 
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.auto import tqdm
import torch.optim as optim

from numpy import float32



#Utils

In [5]:
def save_checkpoint(state, filename="classification_checkpoint.pth.tar"):
  print("=> Saving checkpoint")
  torch.save(state, filename)

In [6]:
def load_checkpoint(checkpoint_path, model, load_encoder = True, freeze_encoder=False, checkpoint_path_encoder=None):
  print("=> Loading checkpoint")

  if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint["state_dict"])

  elif load_encoder:
    checkpoint = torch.load(checkpoint_path_encoder)
    pretrained_dict = checkpoint["state_dict"]
    model_dict = model.state_dict()
  
    for k, v in pretrained_dict.items():
      if k in model_dict:
        model_dict[k] = v    
      
    # 2. overwrite entries in the existing state dict
    model_dict.update(model_dict) 
    
    # 3. load the new state dict
    model.load_state_dict(model_dict)

  if freeze_encoder:
    ct = 0
    for child in model.children():
      if ct < 2:
          for param in child.parameters():
              param.requires_grad = False
      else:
        break

      ct +=1
      # print(child)
      # print("Step")
  

In [7]:
def get_loaders(
    train_dir,
    val_dir,
    batch_size,
    train_transform,
    val_transform,
    class_dic,
    num_workers=4,
    pin_memory=True,
):
  train_ds = CarvanaDataset(
      image_dir = train_dir,
      class_dic  = class_dic,
      transform = train_transform, 
  )
  train_loader = DataLoader(
      train_ds,
      batch_size=batch_size,
      num_workers=num_workers,
      pin_memory=pin_memory,
      shuffle=True,
  )

  val_ds = CarvanaDataset(
      image_dir = val_dir,
      class_dic  = class_dic,
      transform = val_transform, 
  )
  val_loader = DataLoader(
      val_ds,
      batch_size=batch_size,
      num_workers=num_workers,
      pin_memory=pin_memory,
      shuffle=False,
  )

  return train_loader, val_loader

In [8]:
def check_advance(loader, model, device="cuda"):
  model.eval()
  #To do
  model.train()

In [9]:
def kernel_size(initial_value, iterations, division_value=2):
  for i in range(iterations):
    initial_value//=division_value
  return initial_value

# Model 

In [10]:
class DoubleConv(nn.Module):
  def __init__ (self, in_channels, out_channels):
    super(DoubleConv, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
        #Here we add Batch Normalization to improve the paper's model
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    )

  def forward(self, x):
    return self.conv(x)
      

In [11]:
class UNET_CLASS(nn.Module):
  #features represent the original paper dimensions.
  def __init__(self, in_channels=3, out_channels=5, 
               features=[64, 128, 256, 512],
               linear=[250, 150, 100, 50],
               img_dim=None):
    super(UNET_CLASS, self).__init__()
    
    if img_dim == None:
      raise NameError('Need to put image dimension')
    
    #Encoder part
    self.downs = nn.ModuleList()
    #Pool
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    #Drop_out
    self.drop_out = nn.Dropout()
    #Classification part
    self.linear_layers = nn.Sequential()

    
   #Create a list of contracting path 
    for feature in features:
      self.downs.append(DoubleConv(in_channels, feature))
      in_channels = feature

   
    #Create a list of linear classificator
    linear_count = 1
    activation_count = 1

    # self.linear_layers.add_module(str(linear_count)+"_linear", nn.Linear(160*160*in_channels, linear[0]))
    # self.linear_layers.add_module(str(linear_count)+"_linear", nn.Linear(19660800, linear[0]))
    # print((kernel_size(img_dim[1],len(features))))
    # print((kernel_size(img_dim[0],len(features))))
    # print(in_channels)
    self.linear_layers.add_module(str(linear_count)+"_linear", 
                                  nn.Linear(in_channels*
                                            (kernel_size(img_dim[1],len(features)))*
                                            (kernel_size(img_dim[0],len(features))),
                                            linear[0]))
    self.linear_layers.add_module(str(activation_count)+"_activ", nn.ReLU())

    linear_count+=1
    activation_count+=1

    actual_linear = linear[0]

    for linear_dimesion in linear[0:]:
      self.linear_layers.add_module(str(linear_count)+"_linear", nn.Linear(actual_linear, linear_dimesion))
      self.linear_layers.add_module(str(activation_count)+"_activ", nn.ReLU())

      linear_count+=1
      activation_count+=1
      
      actual_linear = linear_dimesion
    
    self.linear_layers.add_module(str(linear_count)+"_linear", nn.Linear(actual_linear, out_channels))
    self.linear_layers.add_module(str(activation_count)+"_activ", nn.Softmax(dim=1))


  def forward(self, x):
   
    #For move in decoder
    for down in self.downs:
      x = down(x)
      x = self.pool(x)
    
    # print(x.shape)
    x = x.reshape(x.size(0), -1)
    x = self.drop_out (x)

    #For move in classficator
    # print(x.shape)
    x = self.linear_layers(x)

    return x

In [12]:
def test():
  #Batch, Channel, H, W
  x = torch.randn((3,3,160,160))
  model = UNET_CLASS(in_channels=3,out_channels=3, img_dim=(160,160))
  preds = model(x)
  print(x.shape)
  print(preds.shape)

test()

torch.Size([3, 3, 160, 160])
torch.Size([3, 3])


# Train

In [13]:
class CarvanaDataset(Dataset):
  def __init__(self, image_dir, class_dic, transform=None):
    self.image_dir = image_dir
    self.transform = transform
    self.images = []
    self.can_classes = len(class_dic)
    self.class_dic = class_dic
    
    for subdir, dirs, files in os.walk(self.image_dir):
      for filename in files:
        subdirectoryPath = os.path.relpath(subdir, self.image_dir) #get the path to your subdirectory
        filePath = os.path.join(self.image_dir, os.path.join(subdirectoryPath, filename)) #get the path to your file
        self.images.append([filePath, subdirectoryPath])

  def __len__(self):
    return len(self.images)
  
  def __getitem__(self, index):
     img_path = self.images[index][0]
    #  print( self.images[index])
    #  print( self.can_classes)

     #The image input is an RBG but the image mask is in a grayscale
     image = np.array(Image.open(img_path).convert("RGB"))
     
    #  label = np.zeros(self.can_classes)
    #  print(label.shape)
    #  print(label)
    #  label[self.class_dic[self.images[index][1]]] = 1
    #  print(label)
     label = np.array(self.class_dic[self.images[index][1]])
     
     #mask[mask == 255.0] = 1.0
     if self.transform is not None:
       augmentations = self.transform(image=image)
       image = augmentations["image"]
     return image, label

# Hyperparameters

In [14]:
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 150
NUM_EPOCHS = 1000
NUM_WORKERS = 0
NUM_CHANNELS = 3
NUM_CLASSES = 10
IMAGE_HEIGHT = 160 #Originally 1280
IMAGE_WIDTH = 240 #Originally 1918
PIN_MEMORY = True
LOAD_MODEL = True
FREEZE_ENCODER = False
LOAD_ENCODER = False
#TRAIN_IMG_DIR = "/content/drive/MyDrive/plants/train"
#TRAIN_MASK_DIR = "/content/drive/MyDrive/plants/train_masks"
#VAL_IMG_DIR = "/content/drive/MyDrive/plants/val"
#VAL_MASK_DIR = "/content/drive/MyDrive/plants/val_masks"

# TRAIN_IMG_DIR = "G:/Mi unidad/plants/train"
# TRAIN_MASK_DIR = "G:/Mi unidad/plants/train_masks"
# VAL_IMG_DIR = "G:/Mi unidad/plants/val"
# VAL_MASK_DIR = "G:/Mi unidad/plants/val_masks"

TRAIN_IMG_DIR = "/content/classification/vechicles/train"
TRAIN_MASK_DIR = "/content/classification/vechicles/train"
VAL_IMG_DIR = "/content/classification/vechicles/test"
VAL_MASK_DIR = "/content/classification/vechicles/test"


#TRAIN_IMG_DIR = "G:/Mi unidad/cavana/train"
#TRAIN_MASK_DIR = "G:/Mi unidad/cavana/train_masks"
#VAL_IMG_DIR = "G:/Mi unidad/cavana/val"
#VAL_MASK_DIR = "G:/Mi unidad/cavana/val_masks"

CLASS_DIC = {
    "bike":0,
    "boat":1,
    "bus":2,
    "car":3,
    "cycle":4,
    "helicopter":5,
    "plane":6,
    "scooty":7,
    "train":8,
    "truck":9
}


print(DEVICE)

cuda


# Training 

In [15]:
#Will going to do 1 epoch training
def train_fn(loader, model, optimizer, loss_fn, scaler):
  #Progress bar
  loop = tqdm(loader)

  for batch_idx, (data, targets) in enumerate (loop):
    data = data.to(device=DEVICE)
    targets = targets.long().to(device=DEVICE)
    
    #Forward
    #Run with mixture precision float 32 ops with float16 and stuffs like that.
    with torch.cuda.amp.autocast():
      predictions = model(data)
      predictions = predictions.float().to(device=DEVICE)

      # print(predictions.shape)
      # print(targets.shape)

      # print(predictions)
      # print(targets)
      loss = loss_fn(predictions, targets)
      
    #Backwards
    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer=optimizer)
    scaler.update()

    #update tqdm loop

    loop.set_postfix(loss=loss.item())

# Main

## Transforms

In [16]:
train_transform = A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    A.Rotate(limit=35, p=1.0),
    A.HorizontalFlip(p=0.1),
    A.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=[1.0, 1.0, 1.0],
        max_pixel_value=255.0,
    ),
    ToTensorV2(),
    #TODO change if is a gray scale image
],additional_targets={'image': 'image'})

val_transform = A.Compose([
    A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    A.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=[1.0, 1.0, 1.0],
        max_pixel_value=255.0,
    ),
    ToTensorV2(),
    #TODO
],additional_targets={'image': 'image'})

## Create model

In [17]:
model = UNET_CLASS(in_channels=NUM_CHANNELS,
                   out_channels=NUM_CLASSES,
                   img_dim=(IMAGE_HEIGHT, IMAGE_WIDTH)).to(DEVICE)
#With Logits because the model has not activation function
loss = nn.CrossEntropyLoss()

In [18]:
#Optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [19]:
train_loader, val_loader = get_loaders(
    TRAIN_IMG_DIR,
    VAL_IMG_DIR,
    BATCH_SIZE,
    train_transform,
    val_transform,
    CLASS_DIC,
    NUM_WORKERS,
    PIN_MEMORY,
)

In [20]:
if LOAD_MODEL:
        load_checkpoint("classification_checkpoint.pth.tar", 
                        model, 
                        LOAD_ENCODER, 
                        FREEZE_ENCODER, 
                        "autoencoder_checkpoint.pth.tar")

=> Loading checkpoint


In [21]:
scaler = torch.cuda.amp.GradScaler()
for epoch in range(NUM_EPOCHS):
  train_fn(train_loader, model, optimizer, loss, scaler)

  checkpoint = {
      "state_dict"  : model.state_dict(),
      "optimizer"   : optimizer.state_dict()
  }

  save_checkpoint(checkpoint)

  #Check acc

  check_advance(val_loader, model, device=DEVICE)

#Loader


=> Saving checkpoint



=> Saving checkpoint



=> Saving checkpoint



=> Saving checkpoint



=> Saving checkpoint


...


=> Saving checkpoint



=> Saving checkpoint



=> Saving checkpoint



=> Saving checkpoint


KeyboardInterrupt: ignored